In [1]:
import pandas as pd
import importlib  

In [2]:
%%time
#import funkcji
nb = importlib.import_module("ipynb.fs.defs.michal_battery_stats")

CPU times: user 5.55 ms, sys: 275 µs, total: 5.83 ms
Wall time: 6.32 ms


## proste funkcje

In [3]:
michal_func=nb.michal_func
michal_func2=nb.michal_func2

In [4]:
# it should return an empty DataFrame if the battery was never recharged
assert michal_func(1,2) == 3
assert michal_func(2,2) == 4

In [5]:
# it should return an empty DataFrame if the battery was never recharged
assert michal_func2(1,2) == -1
assert michal_func2(2,2) == 0

## Na podstawie stworzonych danych tymczasowych

In [6]:
remove_consecutive_same_values = nb.remove_consecutive_same_values

In [8]:
def build_battery_history(values, device_id="60df87b4be3fc900069dac1f", start_time=1600000000000):
    id_array = [device_id] * len(values)
    values30 =  [i-30  for i in values]
    values2 = [i+2  for i in values]
    timestamp_array = map(lambda t: start_time + t * 10000, range(len(values)))
    return pd.DataFrame({
        "ID": pd.Series(id_array),
        "battery": pd.Series(values, dtype="int64"),
        "battery-30": pd.Series(values30, dtype="int64"),
        "battery+2": pd.Series(values2, dtype="int64"),
        "Timestamp": pd.Series(timestamp_array, dtype="int64")
    })

In [9]:
# it keeps a single value
df_single_value = build_battery_history([25])
remove_consecutive_same_values( df_single_value )
assert len(df_single_value) == 1

In [10]:
# it removes a duplicate value
df_duplicate_values = build_battery_history([25,25]) 
assert len( remove_consecutive_same_values( df_duplicate_values ) ) == 1

In [11]:
# it does not remove non-consecutive same values
df_non_consecutive_same = build_battery_history([25,50,25]) 
assert len( remove_consecutive_same_values( df_non_consecutive_same ) ) == 3

In [12]:
# it removes multiple duplicates
df_multiple_duplicates = build_battery_history([25,50,50,20,20]) 
assert len( remove_consecutive_same_values( df_multiple_duplicates ) ) == 3

In [13]:
# it ignores consecutive values when they are from different devices
df_device_a = build_battery_history([25,50])
df_device_b = build_battery_history([50,75],device_id="70df87b4be3fc900069ddd9")
df_combined = pd.concat([df_device_a,df_device_b])
assert len(remove_consecutive_same_values( df_combined )) == 4

## na podstawie pliku

In [14]:
load_data = nb.load_data
battery_minus_30 = nb.battery_minus_30
battery_plus_2 = nb.battery_plus_2

In [15]:
df = load_data()
df.head()

,ID,Timestamp,battery,battery-30,battery+2
0,60df87b4be3fc900069dac1f,1625262191000,100,70,102
1,60df87b4be3fc900069dac20,1625262191000,100,70,102
2,60df87b4be3fc900069dac21,1625262191000,100,70,102
3,60df87b4be3fc900069dac22,1625262191000,100,70,102
4,60df87b4be3fc900069dac1f,1625262311000,100,70,102


In [16]:
for index, row in df.iterrows():
    assert battery_minus_30(row.battery) == row['battery-30']
    assert battery_plus_2(row.battery) == row['battery+2']

## TODO

1. info na temat rekordu w którym jest błąd
2. zewnętrzne odpalanie wszystkich testów z linii komend
3. importowanie funkcji z innych katalogów
4. research odnośnie tagów chunkow

## PO CO

1. aby testy były w innych plikach/odseparowane od funkcji i ich wykorzystania
2. aby spisywać testy w plikach
3. aby można było wykonać wszystkie testy wszystkich użytkowników
4. aby testy były wykonywane przy każdym commicie na bazę automatycznie
5. aby każdy był odpowiedzialny za to, iż swoją zmianą nie wywala jakichś testó
